### Kaggle BaseLine

In [98]:
#analyse
import pandas as pd
import numpy as np


# prepare data
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.utils import shuffle


#processing
from sklearn.pipeline import Pipeline


#Quality control and metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


#Models
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection,linear_model, preprocessing, ensemble, metrics, svm
from sklearn.linear_model import LogisticRegression
import lightgbm


#scaller
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer

#encoder
import category_encoders as ce
from category_encoders.cat_boost import CatBoostEncoder


#fill nan
from sklearn.impute import SimpleImputer


## visualise
import seaborn as sns


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [159]:
test_data = pd.read_csv('/Users/nikli/Documents/git_reps/Data/orange_small_churn_test_data.csv',index_col = 'ID')
train_data = pd.read_csv('/Users/nikli/Documents/git_reps/Data/orange_small_churn_train_data.csv',index_col = 'ID')
train_labels=train_data['labels'].fillna(-1)
train_data['labels']=train_data['labels'].fillna(-1)
train_data=train_data.drop('labels', axis=1)
train_data.loc[:,'Var190':].head()

,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NESt0G8EIb,AERks4l,NaN,taul,1K8T,0LaQ,UaKK0yW,I1sFbv_0IT,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,YFAj,Bnunsla,o64y9zI,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,NaN,FoxgUHSK8h,LrdZy8QqgUfkVShG,NaN,taul,1K8T,TyGl,fhk21Ss,nQUveAzAF7,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,NaN,vNEvyxLp3Z,RO12,NaN,taul,1K8T,0Xwj,uoZk2Zj,LWyxgtXeJL,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,4e7gUH7IEC,RO12,NaN,taul,1K8T,vSNn,kugYdIL,ZIXKpoNpqq,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


## План:
Построить базовые модели на основе преобразования датасета различными способами и замерить качество базовых моделей:
<!-- - <a href='#fillna'> 1. Закодировать категориальные признаки,заменить пропуски </a>
- <a href='#first_result'> 2. Сделать просто фит предикт - записать результат модели на сабмите </a>
- <a href='#prepare_dataset'> 3. Сделать анализ датафрейма, удалить признаки, которые слабо коррелируют - записать результат модели на сабмите </a>
- <a href='#prepare_dataset'> 6. Сделать кросс валидацию & подбор параметров модели - записать результат модели на сабмите  </a>
- <a href='#prepare_dataset'> 7. Сделать вывод про лучший скор </a>
 -->

#### Удалим столбцы тестовой выборки,в которых нет никакой информации : set(column) = {Nan}

In [160]:
print('Способ поиска пропусков')
print(train_data['Var79'].unique())
print('Так мы будем находить массив уникальные значения ,которые являются рпопусками или нет')
print([np.isnan(nan) for nan in train_data['Var79'].unique()])
print('Проверим на наличие пропусков')
print(any([np.isnan(nan) for nan in train_data['Var79'].unique()]))
array=[np.isnan(nan) for nan in train_data['Var78'].unique()]
print('Мы должны добавить ещё одну проверку , на длинну массива')
print(array)
print(False in array)

Способ поиска пропусков
[nan]
Так мы будем находить массив уникальные значения ,которые являются рпопусками или нет
[True]
Проверим на наличие пропусков
True
Мы должны добавить ещё одну проверку , на длинну массива
[False, True, False, False, False, False, False, False, False, False, False, False, False]
True


#### Удалим неинформативные признаки

In [161]:
def undersampling(data, labels,col_to_label):

    data['Labels']=labels
    dataChurn = data[data['Labels'] == 1]
    
    dataNoChurn = data[data['Labels'] == -1]
    
    countFirst = len(dataChurn)
    dataChurn = dataChurn.append(dataNoChurn.iloc[:countFirst,:])
    dataChurn = shuffle(dataChurn)
    return dataChurn.iloc[:,:col_to_label], dataChurn.iloc[:,col_to_label:]
train_data , train_labels = undersampling(train_data, train_labels, 230)

# def dropUninformative(data):
#     df=data.copy()
#     imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#     df.loc[:,:] = imputer.fit_transform(df.loc[:,:])
#     return data[df.loc[:,  (df != df.iloc[0]).any()].columns]

def dropUninformative(data):
    data=data.copy()
    df = pd.DataFrame([])
    for i in range(data.shape[1]):
        if len(data.iloc[:,i].unique())>1:
            df["Var"+str(i+1)] = data.iloc[:,i]
    return df


train_data=dropUninformative(train_data)
test_data=test_data[train_data.columns]


In [162]:
### Теперь мы можем тестировать способы замены пропусков в числах:
### Теперь мы можем тестировать способы кодирования данных:
def fill_numeric(train_data,  strategy="mean"):
    train_data=train_data.copy()
    if strategy=="mean":
        imputer = SimpleImputer(missing_values=np.nan, strategy=strategy)

    if strategy=='median':
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')

    if strategy=='most_frequent':
        imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    if strategy=='0':
        train_data=train_data.fillna(0)
        return train_data

    train_data.loc[:,:] = imputer.fit_transform(train_data.loc[:,:])
    return train_data


fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="mean")
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="median")
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="most_frequent")
fill_numeric(train_data[train_data.columns.difference(train_data.loc[:,'Var191':'Var229'].columns)],  strategy="0")

,Var1,Var10,Var100,Var101,Var102,Var103,Var104,Var105,Var106,Var107,...,Var90,Var91,Var92,Var93,Var94,Var95,Var96,Var97,Var98,Var99
ID,,,,,,,,,,,,,,,,,,,,,
12717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,53031.0,0.0,0.0,0.0,0.0,0.0
559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
### Теперь мы можем тестировать способы замены пропусков в числах:
### Теперь мы можем тестировать способы кодирования данных:
def fill_cat(train_data,  strategy="most_frequent"):
    train_data=train_data.copy()
    
    if strategy=='most_frequent':
        imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        train_data.loc[:,:] = imputer.fit_transform(train_data.loc[:,:])
        #print('most frequent')
        
    if strategy=='No_info':
        train_data=train_data.fillna('No_info')
        #print('Done No_info!')
    
    if strategy=='bfill':
        train_data=train_data.fillna(method='bfill')
        train_data=train_data.fillna('No_info')
        #print('Done method bfill!')
        
    if strategy=='pad':
        train_data=train_data.fillna(method='pad')
        train_data=train_data.fillna('No_info')
        #print('Done method pad!')    


    return train_data

fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="most_frequent")
fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="No_info")
fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="bfill")
fill_cat(train_data[train_data.columns.difference(train_data.loc[:,'Var1':'Var190'].columns)],  strategy="pad")

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
ID,,,,,,,,,,,,,,,,,,,,,
12717,No_info,CEat0G8rTN,2Knk1KF,SEuy,taul,1K8T,wOc1,fayYfhR,YQcPlrpk3ptPA,UsLnYKt,...,KbkKEj0,zCkv,QqVuch3,LM8l689qOp,No_info,kG3k,fKCe,02N6s8f,Zy3gnGM,am7c
12410,No_info,4e7gUH7IEC,RO12,SEuy,taul,1K8T,0Xwj,ho_bxgw,TP5nuUTLjIwLuWf8pjH,UsLnYKt,...,AvcOVQC,oslk,r757j0h,jySVZNlOJy,No_info,kG3k,w_Ub,RAYp,F2FyR07IdsN7I,am7c
779,No_info,a4vPe2fHUn,RO12,SEuy,taul,1K8T,PGNs,XPJvY8M,1lFJzqR,hAV35Zc,...,nNRvOIl,oslk,VVjqhZX,jySVZNlOJy,No_info,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86
15965,No_info,J9Vr4RQZiT,2Knk1KF,SEuy,taul,1K8T,kNzO,FJD_9E2,o64y9zI,CJCmkSe,...,ylCK4WV,oslk,kiBRltV,jySVZNlOJy,No_info,kG3k,FSa2,6fzt,Zy3gnGM,mj86
476,No_info,AUUt0GoOh3,RO12,SEuy,taul,1K8T,7gSz,tAqBboK,nQUq7hGe64,W0mqXyJ,...,B0LABPr,oslk,UP3EVaf,LM8l689qOp,No_info,kG3k,Qu4f,RAYp,F2FyR07IdsN7I,am7c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,r__I,gOyrjIs7u7,RO12,SEuy,taul,1K8T,ssAy,6KF0k8W,9g4oaYjoL0,2rMgZjZ,...,ot6oLzk,oslk,IIvC99a,LM8l689qOp,4n2X,ELof,PM2D,RAYp,F2FyR07IdsN7I,am7c
559,r__I,crIt0Go8Lb,RO12,SEuy,taul,1K8T,wGW5,jCepSrJ,J_sb4dmQwP,2rMgZjZ,...,Oy_RPEi,oslk,FS4qjNq,LM8l689qOp,4n2X,ELof,Qu4f,RAYp,F2FyR07IdsN7I,am7c
16067,r__I,P1WvyxLp3Z,2Knk1KF,SEuy,taul,1K8T,ssAy,8Nsn57c,TP5upuTLjIwLuWf8pjH,ZkcHC5O,...,07m8iqZ,oslk,VjJAgOW,LM8l689qOp,4n2X,xG3x,xb3V,RAYp,55YFVY9,mj86


#### Закодируем данные

In [297]:
def cat_coding(train_data, train_labels,  method, test_data):
    train_data=train_data.copy()
    train_data['labels'] = train_labels
    cols=train_data.loc[:,'Var191':'Var229'].columns
    
    if method=='frequency':
        for col in cols:
            freq_encode = train_data.groupby(col).size() / train_data.shape[0]
            train_data.loc[:, col] = train_data[col].map(freq_encode)
            test_data.loc[:, col] = test_data[col].map(freq_encode)
        #train_data= train_data
        
    elif method=='mean':
        for col in cols:
            mean_encode = train_data.groupby(col)['labels'].mean() # Считаем среднее значение таргета группиря по нашему столбцу
            train_data.loc[:, col] = train_data[col].map(mean_encode)
            test_data.loc[:, col] = test_data[col].map(mean_encode)
        
    elif method=='smoothed':
        for col in cols:
            mean = train_data['labels'].mean() # Находим среднее значение
            agg = train_data.groupby(col)['labels'].agg(['count', 'mean']) # Считаем среднее значение и число повторов значения столбца
            counts = agg['count']
            means = agg['mean']
            weight = 100
            smooth = (counts * means + weight * mean) / (counts + weight)
            train_data.loc[:, col] = train_data[col].map(smooth)
            test_data.loc[:, col] = test_data[col].map(smooth)
            
            
    elif method=='WOE':#Weight of Evidence encoding | считаем ln отношения вероятностей принадлежности к классам
        WOEE_encoder = ce.WOEEncoder()
        print(train_data.isna().sum().sum())
        print(np.any(np.isnan(train_data)))
        train_data.loc[:,'Var191':'Var229'] = WOEE_encoder.fit_transform(train_data.loc[:,'Var191':'Var229'],
                                                                         train_data['labels'])
        test_data.loc[:,'Var191':'Var229'] = WOEE_encoder.transform(test_data.loc[:,'Var191':'Var229'])
    elif method=='leave_one_out':#Leave-one-out encoding | Считает среднее значение таргета для случая, когда пример удален из дата-сета.
        LOOE_encoder = ce.LeaveOneOutEncoder()
        train_data.loc[:,'Var191':'Var229'] = LOOE_encoder.fit_transform(train_data.loc[:,'Var191':'Var229'],
                                                                         train_data['labels'])
        test_data.loc[:,'Var191':'Var229'] = LOOE_encoder.transform(test_data.loc[:,'Var191':'Var229'])

    elif method=='сatboost':
        CBE_encoder = CatBoostEncoder()
        train_data.loc[:,'Var191':'Var229'] = CBE_encoder.fit_transform(train_data.loc[:,'Var191':'Var229'],
                                                                         train_data['labels'])
        test_data.loc[:,'Var191':'Var229'] = CBE_encoder.transform(test_data.loc[:,'Var191':'Var229'])

    pass
    del train_data['labels']
    return train_data, test_data
        


In [280]:
def results_append (model : str, cat_cod : str, fill_num : str, fill_cat : str,  x_train, x_test, y_train, y_test):
    if model=='CatBoost':
        clf=CatBoostClassifier(iterations=90)
        clf.fit(x_train, y_train, verbose=False)
        model_info = {
                 'Model' : model, 
                 'Cat_coding' : cat_cod,
                 'Num_fill' : fill_num,
                 'Cat_fill' : fill_cat,
                 'Train auc score' : round(roc_auc_score(y_train, clf.predict_proba(x_train)[:, 1]), 3), 
                 'Train f1 score' : round(f1_score(y_train, clf.predict(x_train)), 3),
                 'Test auc score' : round(roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]), 3), 
                 'Test f1 score' : round(f1_score(y_test, clf.predict(x_test)), 3)
                 }
        return model_info
    elif model=='XGBoost':
        clf=XGBClassifier()
    elif model=='Logistsic Regression':
        clf=LogisticRegression()
    elif model=='LightGBM':
        clf=lightgbm.LGBMClassifier()
    elif model=='Decision Tree':
        clf=DecisionTreeClassifier()
    elif model=='Random Forest':
        clf=ensemble.RandomForestClassifier()
        
    
    clf.fit(x_train, y_train)
    model_info = {
                 'Model' : model, 
                 'Cat_coding' : cat_cod,
                 'Num_fill' : fill_num,
                 'Cat_fill' : fill_cat,
                 'Train auc score' : round(roc_auc_score(y_train, clf.predict_proba(x_train)[:, 1]), 3), 
                 'Train f1 score' : round(f1_score(y_train, clf.predict(x_train)), 3),
                 'Test auc score' : round(roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]), 3), 
                 'Test f1 score' : round(f1_score(y_test, clf.predict(x_test)), 3)
                 }
    return model_info


In [281]:
cat_fill_methods=['most_frequent', 'No_info', 'bfill', 'pad']
num_fill_methods=['mean', 'median', 'most_frequent', '0']
cat_code_methods=['frequency', 'mean', 'smoothed', 'leave_one_out', 'сatboost']
models=['CatBoost', 'XGBoost', 'Logistsic Regression', 'LightGBM', 'Decision Tree', 'Random Forest']

In [282]:
result = pd.DataFrame({'Model' : [], 'Cat_coding':[], 'Num_fill':[], 'Cat_fill':[], 'Train auc score' : [],
                       'Train f1 score' : [], 'Test auc score' : [], 'Test f1 score' : []})
result

,Model,Cat_coding,Num_fill,Cat_fill,Train auc score,Train f1 score,Test auc score,Test f1 score


In [290]:
%%time
for cat_meth in cat_fill_methods:
    for num_meth in num_fill_methods:
        for cat_cod_meth in cat_code_methods:
            for model in models:
                train_data_exp = train_data.copy()

                train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var191':'Var229'].columns)]=fill_numeric(
                    train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var191':'Var229'].columns)],
                    strategy=num_meth)
                
                train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var1':'Var190'].columns)]=fill_cat(
                    train_data_exp[train_data_exp.columns.difference(train_data_exp.loc[:,'Var1':'Var190'].columns)],
                    strategy=cat_meth)
                
                y =train_data_exp['labels']
                del train_data_exp['labels']
                X = train_data_exp
                x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345678)

                x_train, x_test=cat_coding(x_train, y_train, cat_cod_meth, x_test)
                x_test=x_test.fillna(method='pad')
                
                #print(model,cat_cod_meth,num_meth,cat_meth)
                
                info = results_append (model=model, cat_cod=cat_cod_meth, fill_num=num_meth,
                                fill_cat=cat_meth, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)
                result=result.append(info ,ignore_index=True)

Wall time: 11min 38s


#### Было немного проблемматично записать все результаты:
1. Изначально запись глобального датафрейма была из функции 'results_append', что для норм кода не ок
2. Неправильно кодировал категориальные признаки. Я кодировал весь датасет в разбивке по таргету. Из-за этого были сформированны линейно-зависимые с таргетом признаки.
3. Обучающая и тестовая выборка не всегда одинаковые, если какого-то значения не было в обучающей выборке, то в тестовой это значение заменится на Nan. Такой пропуск был лишь 1 , заполнил его предыдущим значением.


In [288]:
result.sort_values(by=['Test auc score','Test f1 score'],ascending=False).head(20)

,Model,Cat_coding,Num_fill,Cat_fill,Train auc score,Train f1 score,Test auc score,Test f1 score
18,CatBoost,leave_one_out,mean,most_frequent,0.955,0.879,0.721,0.685
24,CatBoost,сatboost,mean,most_frequent,0.955,0.879,0.721,0.685
384,CatBoost,сatboost,mean,pad,0.964,0.899,0.720,0.697
144,CatBoost,сatboost,mean,No_info,0.964,0.898,0.720,0.679
145,XGBoost,сatboost,mean,No_info,0.911,0.823,0.719,0.679
264,CatBoost,сatboost,mean,bfill,0.965,0.902,0.718,0.698
205,XGBoost,сatboost,most_frequent,No_info,0.906,0.826,0.717,0.692
414,CatBoost,сatboost,median,pad,0.960,0.893,0.716,0.681
234,CatBoost,сatboost,0,No_info,0.974,0.914,0.714,0.673
354,CatBoost,сatboost,0,bfill,0.972,0.913,0.713,0.687


In [296]:
print('model results')
display(result.groupby('Model')['Test auc score','Test f1 score'].mean().sort_values(by=['Test auc score','Test f1 score'],ascending=False))

print('model best results')
display(result.groupby('Model')['Test auc score','Test f1 score'].max().sort_values(by=['Test auc score','Test f1 score'],ascending=False))

print('Cat coding results')
display(result.groupby('Cat_coding')['Test auc score','Test f1 score'].mean().sort_values(by=['Test auc score','Test f1 score'],ascending=False))

print('Num fill results')
display(result.groupby('Num_fill')['Test auc score','Test f1 score'].mean().sort_values(by=['Test auc score','Test f1 score'],ascending=False))

print('Cat fill results')
display(result.groupby('Cat_fill')['Test auc score','Test f1 score'].mean().sort_values(by=['Test auc score','Test f1 score'],ascending=False))


model results


,Test auc score,Test f1 score
Model,,
CatBoost,0.669888,0.656263
XGBoost,0.669200,0.659600
Random Forest,0.663788,0.646213
LightGBM,0.648100,0.644825
Logistsic Regression,0.554650,0.585725
Decision Tree,0.554587,0.581200


model best results


,Test auc score,Test f1 score
Model,,
CatBoost,0.721,0.698
XGBoost,0.719,0.692
Random Forest,0.711,0.683
LightGBM,0.703,0.690
Decision Tree,0.607,0.661
Logistsic Regression,0.565,0.598


Cat coding results


,Test auc score,Test f1 score
Cat_coding,,
сatboost,0.653453,0.641594
frequency,0.639276,0.620448
smoothed,0.637677,0.619995
mean,0.635891,0.618599
leave_one_out,0.567214,0.644219


Num fill results


,Test auc score,Test f1 score
Num_fill,,
mean,0.629075,0.630525
median,0.628658,0.628000
most_frequent,0.624675,0.629983
0,0.624400,0.627375


Cat fill results


,Test auc score,Test f1 score
Cat_fill,,
most_frequent,0.642829,0.623604
No_info,0.625654,0.634604
pad,0.621888,0.627808
bfill,0.616437,0.629867


#### Результаты: 
1. Лучашя дефолтная модель - CatBoost ( Подтвердил слова создателей )
2. Лучший способ кодированя - CatBoost
3. Лучшее способ заполнения численных пропуском - mean
4. Лучшее способ заполнения численных пропуском - most_frequent
5. Большая часть деревьев(Decision Tree) переобучалась - ничего удивительного
6. Логистическая регрессия показала себя хуже всех
7. XGBoost дал почти такой же результат, как и CatBoost - поэтому его и выбираем, его можно сильнее дообучить!